<a href="https://colab.research.google.com/github/abishekabi/Volcanic-Eruption-Prediction/blob/main/predict_volcanic_eruptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd 'gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions'

/content/gdrive/My Drive/Colab Notebooks/predict-volcanic-eruptions


In [3]:
# !wget "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/19059/1581061/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603216878&Signature=Xcz3Ni8%2F7bb7rIyQlyeyfbeASoi78143MpaVMe3kyZXi3WKfxcW7kp%2F44CQYzgoE2FfwzGR343oh5QVkHTt9xTGjHufJZRxbiJ0allxJiMFJrdZaZLZP8UwcQGl4k7ZlyF%2Bc7lAcoYNI14LjW4GEUSTOYvWyIrJqWGsBPwYCcLaGLaHP4lKxF0W7M%2FIOlyIMASugpiRIVsuCkR8fNWT3k4vyRgSldLTlfI46fkNvOFIsOl8kU%2BeRVd8yuhBrA8EmY8QRR0%2FGl9wSCfyjQe8Or4nSWs5V5svpRrzbtl5gTeMcIi8emEAktsbg8lK3%2F%2BzpIMu4map2I1aXj1wAqMPnYA%3D%3D&response-content-disposition=attachment%3B+filename%3Dpredict-volcanic-eruptions-ingv-oe.zip"
# !unzip data.zip
# from google.colab import files
# files.upload() 
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json  # set permission
#!kaggle competitions download -c predict-volcanic-eruptions-ingv-oe 

In [4]:
!ls -lh 

total 9.5G
-rw------- 1 root root 9.5G Oct 17 18:06 data.zip
-rw------- 1 root root  22K Oct 26 20:37 predict-volcanic-eruptions.ipynb
-rw------- 1 root root  56K Oct 17 18:08 sample_submission.csv
drwx------ 2 root root 4.0K Oct 17 18:06 test
drwx------ 2 root root 4.0K Oct 19 15:53 train
-rw------- 1 root root  84K Oct 19 16:30 train.csv


In [5]:
!head train.csv

segment_id,time_to_eruption
1136037770,12262005
1969647810,32739612
1895879680,14965999
2068207140,26469720
192955606,31072429
1640671020,22264866
288840453,34952168
1162441568,9504818
1791400073,11719563


In [6]:
!head train/1136037770.csv

sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
220.0,339.0,-336.0,364.0,,492.0,-796.0,-423.0,-582.0,-40.0
178.0,221.0,-317.0,366.0,,525.0,-754.0,-415.0,-932.0,-305.0
151.0,42.0,-280.0,250.0,,463.0,-772.0,-229.0,-257.0,-59.0
162.0,-123.0,-243.0,288.0,,303.0,-899.0,212.0,-295.0,38.0
158.0,-287.0,-300.0,372.0,,169.0,-769.0,755.0,169.0,-239.0
136.0,-302.0,-308.0,242.0,,241.0,-510.0,888.0,282.0,-293.0
124.0,-131.0,-292.0,132.0,,234.0,-494.0,970.0,-326.0,-203.0
127.0,133.0,-208.0,14.0,,134.0,-748.0,700.0,237.0,-278.0
142.0,374.0,-81.0,-177.0,,-200.0,-678.0,402.0,664.0,-460.0


In [7]:
!head sample_submission.csv

segment_id,time_to_eruption
1000213997,0
100023368,0
1000488999,0
1001028887,0
1001857862,0
1003310246,0
1003545497,0
1004288653,0
10044968,0


In [8]:
!head test/1000213997.csv

sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10
180.0,,295.0,-221.0,-81.0,379.0,240.0,-288.0,152.0,0.0
200.0,,199.0,-99.0,32.0,-226.0,241.0,-114.0,36.0,0.0
193.0,,113.0,17.0,164.0,52.0,294.0,27.0,-21.0,0.0
188.0,,26.0,125.0,274.0,-278.0,324.0,338.0,-22.0,0.0
177.0,,-48.0,183.0,355.0,-363.0,356.0,339.0,-32.0,0.0
192.0,,-123.0,206.0,414.0,338.0,322.0,493.0,-59.0,0.0
198.0,,-187.0,224.0,452.0,-3.0,257.0,360.0,-30.0,0.0
202.0,,-227.0,221.0,450.0,845.0,208.0,212.0,2.0,0.0
211.0,,-292.0,235.0,388.0,-422.0,158.0,92.0,-4.0,0.0


In [9]:
!ls

data.zip			  sample_submission.csv  train
predict-volcanic-eruptions.ipynb  test			 train.csv


# Data

In [10]:
import gc
import glob
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm
import seaborn as sns

In [11]:
train = pd.read_csv("train.csv")
sample_submission = pd.read_csv("sample_submission.csv")


In [12]:
train_segment_csvs = glob.glob("train/*")
test_segment_csvs = glob.glob("test/*")

In [13]:
train_segment_csvs[0:5]

['train/579307667.csv',
 'train/579908624.csv',
 'train/579683001.csv',
 'train/580262567.csv',
 'train/581069262.csv']

In [14]:
print("Number of files under train/ is:", len(train_segment_csvs))
print("Number of files under test/ is:", len(test_segment_csvs))

duplicated_segment_id = [segment_id for segment_id in [ test_segment_id.split("/")[-1] for test_segment_id in test_segment_csvs]
                         if (segment_id in [ train_segment_id.split("/")[-1] for train_segment_id in train_segment_csvs])]

print("[Duplicates] Segment ids both in train and test are:", duplicated_segment_id)

Number of files under train/ is: 4431
Number of files under test/ is: 4520
[Duplicates] Segment ids both in train and test are: []


## Load Data

In [39]:
column_names = ["sensor_1", 	"sensor_2", 	"sensor_3", 	"sensor_4", 	"sensor_5", 	"sensor_6", 	"sensor_7", 	"sensor_8", 	"sensor_9", 	"sensor_10"]
column_stats = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
columns = [ i+"_"+j for i in column_names for j in column_stats]
train_data= pd.DataFrame(columns=columns)
# display(train_data.columns)
for train_segment_csv in train_segment_csvs[:100]:
  train_csv = pd.read_csv(train_segment_csv)
  row = []
  for feature in train_csv.columns:
    row.extend(train_csv[feature].describe().values)
  train_data.loc[len(train_data), :] = row
display(train_data.head())


,sensor_1_count,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_2_count,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_25%,sensor_2_50%,sensor_2_75%,sensor_2_max,sensor_3_count,sensor_3_mean,sensor_3_std,sensor_3_min,sensor_3_25%,sensor_3_50%,sensor_3_75%,sensor_3_max,sensor_4_count,sensor_4_mean,sensor_4_std,sensor_4_min,sensor_4_25%,sensor_4_50%,sensor_4_75%,sensor_4_max,sensor_5_count,sensor_5_mean,sensor_5_std,sensor_5_min,sensor_5_25%,sensor_5_50%,sensor_5_75%,sensor_5_max,sensor_6_count,sensor_6_mean,sensor_6_std,sensor_6_min,sensor_6_25%,sensor_6_50%,sensor_6_75%,sensor_6_max,sensor_7_count,sensor_7_mean,sensor_7_std,sensor_7_min,sensor_7_25%,sensor_7_50%,sensor_7_75%,sensor_7_max,sensor_8_count,sensor_8_mean,sensor_8_std,sensor_8_min,sensor_8_25%,sensor_8_50%,sensor_8_75%,sensor_8_max,sensor_9_count,sensor_9_mean,sensor_9_std,sensor_9_min,sensor_9_25%,sensor_9_50%,sensor_9_75%,sensor_9_max,sensor_10_count,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max
0,60001,-1.72699,480.405,-3785,-275,0,274,5152,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,1.37093,261.397,-1227,-173,0,174,1080,60001,-0.892818,359.476,-1791,-231,0,235,1685,60001,-0.157064,259.3,-1774,-161,0,159,1467,60001,-0.428393,472.857,-2353,-311,0,318,2008,60001,3.40016,478.71,-2605,-313,0,319,3298,60001,-0.603707,612.779,-9700,-373,0,367,8299,60001,0.0389494,326.952,-1409,-210,0,211,1781,60001,-7.8496,713.272,-3184,-463,0,455,3206
1,60001,0.964634,238.391,-1971,-133,0,133,2318,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,4.58317,220.134,-1787,-129,0,130,1918,60001,1.00725,295.605,-2584,-166,0,169,2906,60001,-0.686572,182.987,-3160,-94,0,92,3109,60001,-2.16925,469.562,-3057,-273,0,266,4177,60001,-0.366927,378.552,-2299,-244,0,240,2019,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60001,-0.224813,284.49,-2613,-150,0,148,2893,60001,1.35939,524.9,-5348,-285,0,291,5346
2,60001,10.2972,1080.66,-4329,-699,0,723,4866,60001,0.40256,1388.57,-6228,-909,0,908,9097,60001,-1.32971,1097.42,-4042,-742,0,746,3833,60001,-2.89852,1076.6,-4643,-723,0,723,4805,60001,0.705788,374.756,-1511,-251,0,255,1487,60001,15.376,623.507,-2458,-401,0,431,2340,60001,-16.563,583.054,-3453,-404,0,373,3027,60001,-2.8999,717.68,-6437,-471,0,480,5009,58229,-10.2763,1321.37,-4868,-919,0,883,4572,60001,2.2167,1697.78,-6801,-1159,0,1149,7266
3,60001,4.46966,804.239,-3498,-532,0,552,3402,60001,-7.14606,1081.02,-5292,-697,0,685,5635,60001,1.8943,860.557,-4885,-572,0,578,4286,60001,-5.70147,823.602,-3312,-543,0,546,3010,60001,-1.74022,311.187,-1609,-206,0,198,1515,60001,1.65434,570.719,-2594,-378,0,387,2865,60001,-6.57572,583.402,-2749,-398,0,383,2469,60001,1.54757,676.529,-4059,-445,0,448,4983,58793,-9.01376,939.495,-3722,-630,0,619,3845,60001,38.7179,1407.58,-5006,-911,0,935,6596
4,60001,-40.0521,13089.2,-32767,-4362,0,4338,32767,60001,11.3117,10403,-32767,-3486,0,3450,32767,60001,-54.9712,11277.6,-32767,-3122,0,3013,32767,60001,70.9546,11144.7,-32767,-3442,0,3537,32767,50723,1345.22,10786.8,-32767,-1678.5,0,1965,32767,58966,105.126,7889.61,-32767,-1483,0,1554,32767,60001,1.97365,556.857,-6300,-90,0,94,7504,60001,5.58841,1458.51,-15071,-210,0,216,15069,60001,4.68217,12856.2,-32767,-4232,0,4192,32767,60001,-191.754,16626.4,-32767,-7891,0,7624,32767


In [43]:
train_data.fillna()

25

In [16]:
# train_csv = pd.read_csv(train_segment_csvs[0])
# display(train_csv.sensor_1.describe())
# for i, row in train[:2].iterrows():
#   print("Segment ID: %d \t Time to Eruption: %d" %(row.segment_id, row.time_to_eruption) )
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   #sensor_data = sensor_data.fillna(0)
#   #sensor_data = sensor_data.dropna(axis=1)
#   print(sensor_data.isna().sum())
#   with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(sensor_data.describe())
#   print(sensor_data.info())
#   print("--"*30)
# sensor_data.describe().unstack()
# def calculate_stats(sensor_row):
#   X = pd.DataFrame()
#   for row in sensor_row:
#     out = sensor_row[row].describe().values
#     print(out) 
          
# # display(train_csv.sensor_1.describe().values)
# #   print(sensor_row.describe())
#   #X = pd.DataFrame()
#   # sensor_row.mean()
#   # input_data.sum()
#   # X.loc[ts, f'{sensor_id}_mean']      = signal.mean()
#   # X.loc[ts, f'{sensor_id}_std']       = signal.std()
#   # X.loc[ts, f'{sensor_id}_var']       = signal.var() 
#   # X.loc[ts, f'{sensor_id}_max']       = signal.max()
#   # X.loc[ts, f'{sensor_id}_min']       = signal.min()

# # def basic_statistics(, sensor_id, s):
# #     """Computes basic statistics for the training feature set.
    
# #     Args:
# #         t_X (pandas.DataFrame): The feature set being built.
# #         x (pandas.Series): The signal values.
# #         s (int): The integer number of the segment.
# #         postfix (str): The postfix string value.
# #     Return:
# #         t_X (pandas.DataFrame): The feature set being built.
# #     """
# #     X = pd.DataFrame()
# #     X.loc[sensor_num, f'{sensor_id}_mean']      = signal.mean()
# #     X.loc[sensor_num, f'{sensor_id}_std']       = signal.std()
# #     X.loc[sensor_num, f'{sensor_id}_var']       = signal.var() 
# #     X.loc[sensor_num, f'{sensor_id}_max']       = signal.max()
# #     X.loc[sensor_num, f'{sensor_id}_min']       = signal.min()

# #     X.loc[s, f'{sensor}_sum{postfix}']       = x.sum()
# #     t_X.loc[s, f'{sensor}_mean{postfix}']      = x.mean()
# #     t_X.loc[s, f'{sensor}_std{postfix}']       = x.std()
# #     t_X.loc[s, f'{sensor}_var{postfix}']       = x.var() 
# #     t_X.loc[s, f'{sensor}_max{postfix}']       = x.max()
# #     t_X.loc[s, f'{sensor}_min{postfix}']       = x.min()
# #     t_X.loc[s, f'{sensor}_median{postfix}']    = x.median()
# #     t_X.loc[s, f'{sensor}_skew{postfix}']      = x.skew()
# #     t_X.loc[s, f'{sensor}_mad{postfix}']       = x.mad()
# #     t_X.loc[s, f'{sensor}_kurtosis{postfix}']  = x.kurtosis()

# #     return t_X
# for i, row in train[:1].iterrows():
#   print("Segment ID: %d \t Time to Eruption: %d" %(row.segment_id, row.time_to_eruption) )
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   print(calculate_stats(sensor_data))
#   #sensor_data = sensor_data.fillna(0)
#   #sensor_data = sensor_data.dropna(axis=1)
#   # print(sensor_data.isna().sum())
#   # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#   #   print(sensor_data.describe())
#   # print(sensor_data.info())
#   print("--"*30)
# train
# # train_379022420 = pd.read_csv("train/379022420.csv")
# # train_1002275321 = pd.read_csv("train/1002275321.csv")
# train.head()
# train.info()

# train_final = pd.DataFrame(columns={'segment_id', 'time_to_eruption', 'sensor_1', 'sensor_2', 'sensor_3',
#        'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9',
#        'sensor_10'})
# for i, row in train[0:20].iterrows():
#   sensor_data = pd.read_csv("train/"+ str(row.segment_id) + ".csv")
#   sensor_data['segment_id'] = row.segment_id
#   train1 = train.merge(sensor_data, on='segment_id')
#   # train_1_describe = pd.DataFrame(train1.describe()).reset_index()
#   # train_1_describe['segment_id'] = row.segment_id
#   train_final = pd.concat([train_final, train1], axis=0)
# train_final = train_final.reset_index(drop=True)
# train_final.groupby(['segment_id', 'time_to_eruption']).mean()
# train_final.isna().sum()

In [23]:
for train_segment_csv in train_segment_csvs[0:1]:
  train_csv = pd.read_csv(train_segment_csv)
  display(train_csv.sensor_1.describe().keys())

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], dtype='object')